<a href="https://colab.research.google.com/github/brunobobadilla06/Portfolio-Proyectos/blob/main/Analisis_de_Ventas_Coderhouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 1) Importación de librerías
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

np.random.seed(42)


In [ ]:

ventas = pd.read_csv("/content/ventas_tecnologia.csv")
ventas.head()


In [ ]:

ventas["fecha_venta"] = pd.to_datetime(ventas["fecha_venta"])

ventas["ingreso"] = ventas["precio_unitario"] * ventas["cantidad"]
ventas["costo_total"] = ventas["costo_unitario"] * ventas["cantidad"]
ventas["ganancia"] = ventas["ingreso"] - ventas["costo_total"]

ventas.head()


In [ ]:

# Ingresos totales
ventas["ingreso"].sum()


In [ ]:

# Ventas por categoría
ventas_categoria = ventas.groupby("categoria", as_index=False)["ingreso"].sum()

sns.barplot(data=ventas_categoria, x="categoria", y="ingreso")
plt.title("Ingresos por categoría de producto")
plt.tight_layout()
plt.show()


In [ ]:

# Ventas por canal
ventas_canal = ventas.groupby("canal_venta", as_index=False)["ingreso"].sum()

sns.barplot(data=ventas_canal, x="canal_venta", y="ingreso")
plt.title("Ingresos por canal de venta")
plt.tight_layout()
plt.show()


In [ ]:

ventas["mes"] = ventas["fecha_venta"].dt.to_period("M").dt.start_time

ventas_mensuales = (
    ventas.groupby("mes", as_index=False)
          .agg(ingreso_mensual=("ingreso", "sum"))
)

plt.plot(ventas_mensuales["mes"], ventas_mensuales["ingreso_mensual"])
plt.title("Evolución mensual de ingresos")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

ventas_mensuales["mes_num"] = ventas_mensuales["mes"].dt.month
ventas_mensuales["tendencia"] = np.arange(len(ventas_mensuales))

ventas_mensuales.head()


In [ ]:

X = ventas_mensuales[["mes_num", "tendencia"]]
y = ventas_mensuales["ingreso_mensual"]

tscv = TimeSeriesSplit(n_splits=4)
errores = []

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    modelo = RandomForestRegressor(n_estimators=300, random_state=42)
    modelo.fit(X_train, y_train)

    pred = modelo.predict(X_test)
    errores.append(mean_absolute_error(y_test, pred))

np.mean(errores)


In [ ]:

futuro = pd.DataFrame({
    "mes_num": [1, 2, 3],
    "tendencia": [len(X), len(X)+1, len(X)+2]
})

futuro["ingreso_proyectado"] = modelo.predict(futuro)
futuro
